In [1]:
import os
import math
import os
import pickle
import numpy as np
import mujoco as mj
from typing import Any, Dict, Optional, Tuple, Union
import gymnasium as gym
from rfc_utils.tools import get_expert
from rfc_utils.config import Config


In [2]:
cfg_id = '050100'

cfg = Config(cfg_id, False, create_dirs=False)

cfg


In [3]:
from agent_envs.humanoid_env import HumanoidTemplate


env = HumanoidTemplate(cfg)

env

this is the model_path mocap_v2.xml


Understaing Expert

In [4]:
expert_qpos, expert_meta = pickle.load(open(cfg.expert_traj_file, "rb"))
print(expert_meta)
print(expert_qpos.shape)



{'dt': 0.03333333333333333, 'mocap_fr': 120, 'scale': 0.45, 'offset_z': 0.0, 'cyclic': False, 'cycle_offset': 0.0, 'select_start': 0, 'select_end': 104, 'fix_feet': False, 'fix_angle': False}
(104, 39)


In [5]:
cfg.env_expert_trail_steps

10

In [6]:
env.action_space

Box(-1.0, 1.0, (38,), float32)

In [7]:
env.data.xpos.shape

(24, 3)

In [8]:
root = env.expert['bquat'][0,:4]
print(root)


[ 0.99805859  0.04664509 -0.01407097  0.03879804]


In [9]:

def multi_quat_norm(nq):
    """return the scalar rotation of a N joints"""

    nq_norm = np.arccos(np.clip(abs(nq[::4]), -1.0, 1.0))
    return nq_norm

In [10]:
import mujoco as mj


In [11]:
n_joint =env.data.qpos.shape[0]//4
body_angvel = np.zeros(n_joint * 3)

print(n_joint)
print(body_angvel.shape)
ind = slice(0,0+4)
#ind
#env.data.qpos.reshape(-1,4)


9
(27,)


In [12]:
expert=env.expert



Computing new expert

In [13]:
from some_math.math_utils import *

In [14]:
env.cfg.obs_coord

'root'

In [15]:
def get_expert(expert_qpos, expert_meta, env):
    # Save the old state
    old_state_qpos = env.data.qpos.copy()
    old_state_qvel = env.data.qvel.copy()

    expert = {'qpos': expert_qpos, 'meta': expert_meta}
    feat_keys = {'qvel', 'rlinv', 'rlinv_local', 'rangv', 'rq_rmh',
                 'com', 'head_pos', 'ee_pos', 'ee_wpos', 'bquat', 'bangvel'}
    for key in feat_keys:
        expert[key] = []

    for i in range(expert_qpos.shape[0]):
        qpos = expert_qpos[i]
        env.data.qpos[:] = qpos
        mj.mj_forward(env.model, env.data)  # Advance the simulation to the current state
        
        rq_rmh = de_heading(qpos[3:7])
        ee_pos = env.get_ee_pos(env.cfg.obs_coord)
        ee_wpos = env.get_ee_pos(None)
        bquat = env.get_body_quat()
        com = env.get_com()
        head_pos = env.get_body_com('head').copy()
        
        if i > 0:
            prev_qpos = expert_qpos[i - 1]
            qvel = get_qvel_fd_new(prev_qpos, qpos, env.dt)
            qvel = qvel.clip(-10.0, 10.0)
            rlinv = qvel[:3].copy()
            if env.cfg.obs_coord =='root':
                rlinv_local = transform_vec(qvel[:3].copy(), qpos[3:7])
            else:
                hq = get_heading_q(qpos[3:7])
                rlinv_local = transform_vec(qvel[:3].copy(), hq)
                
            rangv = qvel[3:6].copy()
            expert['qvel'].append(qvel)
            expert['rlinv'].append(rlinv)
            expert['rlinv_local'].append(rlinv_local)
            expert['rangv'].append(rangv)
        
        expert['ee_pos'].append(ee_pos)
        expert['ee_wpos'].append(ee_wpos)
        expert['bquat'].append(bquat)
        expert['com'].append(com)
        expert['head_pos'].append(head_pos)
        expert['rq_rmh'].append(rq_rmh)

    expert['qvel'].insert(0, expert['qvel'][0].copy())
    expert['rlinv'].insert(0, expert['rlinv'][0].copy())
    expert['rlinv_local'].insert(0, expert['rlinv_local'][0].copy())
    expert['rangv'].insert(0, expert['rangv'][0].copy())

    # Get expert body quaternions
    for i in range(1, expert_qpos.shape[0]):
        bangvel = get_angvel_fd(expert['bquat'][i - 1], expert['bquat'][i], env.dt)
        expert['bangvel'].append(bangvel)
    expert['bangvel'].insert(0, expert['bangvel'][0].copy())

    for key in feat_keys:
        expert[key] = np.vstack(expert[key])
    expert['len'] = expert['qpos'].shape[0]
    expert['height_lb'] = expert['qpos'][:, 2].min()
    expert['head_height_lb'] = expert['head_pos'][:, 2].min()
    if expert_meta.get('cyclic', False):
        expert['init_heading'] = get_heading_q(expert_qpos[0, 3:7])
        expert['init_pos'] = expert_qpos[0, :3].copy()

    # Restore the old state
    env.data.qpos[:] = old_state_qpos
    env.data.qvel[:] = old_state_qvel
    mj.mj_forward(env.model, env.data)
    
    return expert

In [16]:
new_expert = get_expert(expert_qpos,expert_meta,env)

In [17]:
types_dict = {key: type(value) for key, value in new_expert.items()}

print(types_dict)

{'qpos': <class 'numpy.ndarray'>, 'meta': <class 'dict'>, 'ee_pos': <class 'numpy.ndarray'>, 'qvel': <class 'numpy.ndarray'>, 'ee_wpos': <class 'numpy.ndarray'>, 'head_pos': <class 'numpy.ndarray'>, 'bangvel': <class 'numpy.ndarray'>, 'rq_rmh': <class 'numpy.ndarray'>, 'rlinv_local': <class 'numpy.ndarray'>, 'rlinv': <class 'numpy.ndarray'>, 'rangv': <class 'numpy.ndarray'>, 'bquat': <class 'numpy.ndarray'>, 'com': <class 'numpy.ndarray'>, 'len': <class 'int'>, 'height_lb': <class 'numpy.float64'>, 'head_height_lb': <class 'numpy.float64'>}


In [18]:
meta_ex = expert['meta']
meta_new_ex = new_expert['meta']

print(meta_ex)
print(meta_new_ex)

{'dt': 0.03333333333333333, 'mocap_fr': 120, 'scale': 0.45, 'offset_z': 0.0, 'cyclic': False, 'cycle_offset': 0.0, 'select_start': 0, 'select_end': 104, 'fix_feet': False, 'fix_angle': False}
{'dt': 0.03333333333333333, 'mocap_fr': 120, 'scale': 0.45, 'offset_z': 0.0, 'cyclic': False, 'cycle_offset': 0.0, 'select_start': 0, 'select_end': 104, 'fix_feet': False, 'fix_angle': False}


In [45]:
def compare_dict_arrays(new_expert, expert, tol=1e-14):
    # Iterate over keys in new_expert
    for key in new_expert.keys():
        if key == 'meta' or key=='len':
            continue  # Skip the 'meta' key since it is a dictionary
        
        # Check if the key exists in both dictionaries
        if key in expert:
            # Compare numpy arrays
            arr1 = new_expert[key]
            arr2 = expert[key]
            
            # Check if arrays are of the same shape
            if arr1.shape != arr2.shape:
                print(f"Shape mismatch in key: {key}, new_expert shape: {arr1.shape}, expert shape: {arr2.shape}")
                continue
            
            # Find differences within the tolerance
            diff_indices = np.where(np.abs(arr1 - arr2) > tol)
            if diff_indices[0].size > 0:
                print(f"Differences found in key: {key}")
                for index in zip(*diff_indices):
                    print(f"  Difference at index {index}: new_expert value = {arr1[index]}, expert value = {arr2[index]}")

In [46]:
compare_dict_arrays(new_expert, expert)

Differences found in key: bangvel
  Difference at index (30, 13): new_expert value = 3.1152444871863545, expert value = 3.1152444871862923
